# Interactive Gradio Chat Demo

Launch a multimodal chatbot inline in this notebook using Gradio + OpenVINO.

**Prerequisites:** Run `model_preparation.ipynb` first to export the quantized models.

In [ ]:
import sys, os

sys.path.insert(0, os.path.abspath(".."))

MODEL_DIR = "../models/qwen2-vl-2b-instruct-int4"

In [ ]:
from app.model_engine import load_model, new_history, OpenVINOStreamer

print(f"Loading model from {MODEL_DIR}...")
streamer = OpenVINOStreamer(load_model(MODEL_DIR))
print("Model loaded.")

In [ ]:
import gradio as gr
from PIL import Image

chat_history = new_history()

def respond(message, _history):
    user_text = message.get("text", "").strip()
    files = message.get("files", [])
    if not user_text and not files:
        yield ""
        return

    image = None
    if files:
        path = files[0] if isinstance(files[0], str) else files[0].get("path", "")
        if path:
            image = Image.open(path)

    prompt = user_text or "Describe this image."
    response = ""

    for token in streamer.stream(chat_history, prompt, image=image):
        response += token
        yield response

    if streamer.metrics:
        metrics = streamer.metrics
        ttft = metrics.get_ttft().mean
        tok_s = metrics.get_throughput().mean
        n_tok = metrics.get_num_generated_tokens()
        yield response + f"\n\n*TTFT: {ttft:.0f}ms · {tok_s:.1f} tok/s · {n_tok} tokens*"

def clear_chat():
    chat_history.clear()

demo = gr.ChatInterface(
    fn=respond,
    multimodal=True,
    title="Qwen2-VL Chat (OpenVINO)",
    description="Multimodal chatbot — simple notebook demo.",
    textbox=gr.MultimodalTextbox(
        file_types=["image"], file_count="single",
        placeholder="Type a message or upload an image...",
    ),
)
with demo:
    demo.chatbot.clear(clear_chat)
    demo.launch(inline=True, share=False)

In [ ]:
chat_history.get_messages()

In [ ]:
# gr.close_all()